# Exercise 5.2 - multiple map plot (M. Hauser)

prepared by M.Hauser

Here, we want to create a plot with multiple regional maps. For this, we will reproduce one of my old plots (Philip et al., [2017](https://link.springer.com/article/10.1007/s00382-017-3759-x), Figure 7).

<img src="../figures/ex5_multiple_regional_maps_M_Hauser.png"  width="500">

It shows two land-atmosphere coupling metrics ($\pi$ and VAC), as well as the anomalies (in standard deviation) of the sensible heat flux minus potential sensible heat flux  (H'−H'p), temperature anomaly (T') and the latent heat flux anomaly (LH') of a high temperature event in 2015 in the Western US.

## Import Modules

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
import mplotutils as mpu

## Load Data

In [ ]:
fN = "./../data/VAC_2015_06.nc"

ds = xr.open_dataset(fN)

# rename T to TS (because T also stands for 'transpose')
ds = ds.rename(T="TS")

# VAC has values everywhere, put NaN where necessary
ds = ds.assign(VAC=ds.VAC.where(~np.isnan(ds.TS)))

ds

`pi` and `VAC` are land-atmosphere coupling metrics, `LH` the latent heat flux, `TS` (surface) temperature, `h` the difference between the sensible heat flux (`SH`) and the potential sensible heat flux (`SHp`)

## Load Natural Earth Data

This is used later to indicate the US states and country borders.

In [ ]:
states = cfeature.NaturalEarthFeature(
    "cultural", "admin_1_states_provinces_lakes", "50m"
)
border = cfeature.NaturalEarthFeature("cultural", "admin_0_countries", "50m")

## Plot Maps

### Exercise
* create a grid of 1 x 5 maps with a Lambert Conformal projection
* for all axes (use a loop)
  * set the extent to `[-122, -115, 30, 50]`
  * add the States and the Countries (`states` and `border`) with `add_feature`, set no `facecolor` and a gray `edgecolor`

In [ ]:
# code here
# f, axs =

# for ax in axs:
#     ax.set_extent(...)
#     ax.add_feature(...)
#     ax.add_feature(...)

### Solution

In [ ]:
f, axs = plt.subplots(1, 5, subplot_kw=dict(projection=ccrs.LambertConformal()))

for ax in axs:

    ax.set_extent([-122, -115, 30, 50], ccrs.PlateCarree())

    ax.add_feature(states, facecolor="none", edgecolor="0.5")
    ax.add_feature(border, facecolor="none", edgecolor="0.5")

## Continuous-color-plots

Let's plot the maps with non-discrete colormaps

### Exercise

* plot the data using the _xarray_ interface
  * `axs[0]` -> plot `ds.pi`
  * `axs[2]` -> plot `ds.h`
  * `axs[3]` -> plot `ds.TS`
  * `axs[4]` -> plot `ds.LH`
* choose colormaps (`cmap`, see [colorbrewer](http://colorbrewer2.org)) and the data range (`vmin`, `vmax`) 

In [ ]:
f, axs = plt.subplots(1, 5, subplot_kw=dict(projection=ccrs.LambertConformal()))

for ax in axs:

    ax.set_extent([-122, -115, 30, 50], ccrs.PlateCarree())

    ax.add_feature(states, facecolor="none", edgecolor="0.5")
    ax.add_feature(border, facecolor="none", edgecolor="0.5")


ax = axs[0]
# ds.pi.pcolormesh(ax=ax, ...)

# ...

### Solution

In [ ]:
f, axs = plt.subplots(1, 5, subplot_kw=dict(projection=ccrs.LambertConformal()))

for ax in axs:
    ax.set_extent([-122, -115, 30, 50], ccrs.PlateCarree())

    ax.add_feature(states, facecolor="none", edgecolor="0.5")
    ax.add_feature(border, facecolor="none", edgecolor="0.5")

ax = axs[0]
ds.pi.plot.pcolormesh(
    ax=ax,
    cmap="Reds_r",
    vmin=0,
    vmax=2,
    transform=ccrs.PlateCarree(),
    add_colorbar=False,
)

ax = axs[2]
ds.h.plot(
    ax=ax,
    cmap="RdBu_r",
    vmin=-2,
    vmax=2,
    transform=ccrs.PlateCarree(),
    add_colorbar=False,
)


ax = axs[3]
ds.TS.plot(
    ax=ax,
    cmap="RdBu_r",
    vmin=-2,
    vmax=2,
    transform=ccrs.PlateCarree(),
    add_colorbar=False,
)


ax = axs[4]
ds.LH.plot(
    ax=ax,
    cmap="RdBu_r",
    vmin=-2,
    vmax=2,
    transform=ccrs.PlateCarree(),
    add_colorbar=False,
)

## Plotting VAC

VAC comes in four categories (a, b, c, d) with two levels each. It is encoded with numbers from 0 to 8:

* 0: No category
* 1: a, small
* 2: a, large
* 3: b, small
* ...
* 8: d, small

### Exercise
* plot `da.VAC`

> Create levels such that the break is in the middle of the category/ level change. Then use `from_levels_and_colors` to create a new colormap (`cmap`) and norm from `levels` and `colors`.

> The colors used below stem from the following colormap on colorbrewer: https://colorbrewer2.org/#type=qualitative&scheme=Paired&n=8


In [ ]:
from matplotlib.colors import from_levels_and_colors

In [ ]:
colors = [
    "#a6cee3",
    "#1f78b4",
    "#fdbf6f",
    "#ff7f00",
    "#fb9a99",
    "#e31a1c",
    "#b2df8a",
    "#33a02c",
]

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.LambertConformal()))
ax.set_extent([-122, -115, 30, 50], ccrs.PlateCarree())
ax.add_feature(states, facecolor="none", edgecolor="0.5")
ax.add_feature(border, facecolor="none", edgecolor="0.5")

# levels = np.arange(...)
# cmap, norm = from_levels_and_colors(levels, colors=colors)
h = ds.VAC.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    add_colorbar=False,
    extend="neither",
    vmax=8.5,
    vmin=0.5,
)

# colorbar
cbar = mpu.colorbar(h, ax, orientation="horizontal", pad=0.05, size=0.05)
cbar.set_ticks([1.5, 3.5, 5.5, 7.5])
cbar.set_ticklabels(["a", "b", "c", "d"])

### Solution

In [ ]:
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.LambertConformal()))
ax.set_extent([-122, -115, 30, 50], ccrs.PlateCarree())
ax.add_feature(states, facecolor="none", edgecolor="0.5")
ax.add_feature(border, facecolor="none", edgecolor="0.5")

levels = np.arange(0.5, 9.5, 1)
cmap, norm = from_levels_and_colors(levels, colors=colors)
h = ds.VAC.plot(
    ax=ax,
    cmap=cmap,
    levels=levels,
    transform=ccrs.PlateCarree(),
    add_colorbar=False,
    extend="neither",
)

# colorbar
cbar = mpu.colorbar(h, ax, orientation="horizontal", pad=0.05, size=0.05)
cbar.set_ticks([1.5, 3.5, 5.5, 7.5])
cbar.set_ticklabels(["a", "b", "c", "d"])

## Colorbars

The next step is to copy the code from the plotting of VAC back to the other code. I have done that for you. I have also gathered some of the common options into dictionaries. So you can now create and add the colorbars. The one for VAC is already copied in.

### Exercise

* add colorbars for axes 0, and 2 to 4
* use `mpu.colorbar`; you will need to set `size` `pad`, `orientation`

In [ ]:
opt = dict(transform=ccrs.PlateCarree(), add_colorbar=False)

opt_anom = dict(
    cmap="RdBu_r", vmin=-2, vmax=2, transform=ccrs.PlateCarree(), add_colorbar=False
)

# ====================
# create axes

f, axs = plt.subplots(1, 5, subplot_kw=dict(projection=ccrs.LambertConformal()))

for ax in axs:
    ax.set_extent([-122, -115, 30, 50], ccrs.PlateCarree())

    ax.add_feature(states, facecolor="none", edgecolor="0.5")
    ax.add_feature(border, facecolor="none", edgecolor="0.5")

# ====================
# plot data

ax = axs[0]
h0 = ds.pi.plot.pcolormesh(ax=ax, cmap="Reds_r", vmin=0, vmax=2, **opt)

ax = axs[1]
levels = np.arange(0.5, 9.5, 1)
cmap, norm = from_levels_and_colors(levels, colors=colors)
h1 = ds.VAC.plot(ax=ax, cmap=cmap, levels=levels, extend="neither", **opt)

ax = axs[2]
h2 = ds.h.plot(ax=ax, **opt_anom)


ax = axs[3]
h3 = ds.TS.plot(ax=ax, **opt_anom)


ax = axs[4]
h4 = ds.LH.plot(ax=ax, **opt_anom)

# ====================
# add colorbars

cbar = mpu.colorbar(h1, axs[1], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks([1.5, 3.5, 5.5, 7.5])
cbar.set_ticklabels(["a", "b", "c", "d"])

### Solution

In [ ]:
opt = dict(transform=ccrs.PlateCarree(), add_colorbar=False)

opt_anom = dict(
    cmap="RdBu_r", vmin=-2, vmax=2, transform=ccrs.PlateCarree(), add_colorbar=False
)

# ====================
# create axes

f, axs = plt.subplots(1, 5, subplot_kw=dict(projection=ccrs.LambertConformal()))

for ax in axs:
    ax.set_extent([-122, -115, 30, 50], ccrs.PlateCarree())

    ax.add_feature(states, facecolor="none", edgecolor="0.5")
    ax.add_feature(border, facecolor="none", edgecolor="0.5")

# ====================
# plot data

ax = axs[0]
h0 = ds.pi.plot.pcolormesh(ax=ax, cmap="Reds_r", vmin=0, vmax=2, **opt)

ax = axs[1]
levels = np.arange(0.5, 9.5, 1)
cmap, norm = from_levels_and_colors(levels, colors=colors)
h1 = ds.VAC.plot(ax=ax, cmap=cmap, levels=levels, extend="neither", **opt)

ax = axs[2]
h2 = ds.h.plot(ax=ax, **opt_anom)


ax = axs[3]
h3 = ds.TS.plot(ax=ax, **opt_anom)


ax = axs[4]
h4 = ds.LH.plot(ax=ax, **opt_anom)

# ====================
# add colorbars

cbar = mpu.colorbar(h0, axs[0], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks(np.arange(0, 2.1, 1))

cbar = mpu.colorbar(h1, axs[1], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks([1.5, 3.5, 5.5, 7.5])
cbar.set_ticklabels(["a", "b", "c", "d"])

cbar = mpu.colorbar(h2, axs[2], axs[-1], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks(np.arange(-2, 2.1, 0.5))

## Labels and gridlines

### Exercise

* add labels for the plot ('a,' 'b', 'c'), as a title (`loc='left'`)
* add the following titles
  * `axs[0]` -> "$\pi$"
  * `axs[1]` -> "VAC"
  * `axs[2]` -> "SH' - SHp'"
  * `axs[3]` -> "T'"
  * `axs[4]` -> "LH'"
* add gridlines using `ax.gridlines` (4° is a good grid distance)

In [ ]:
opt = dict(transform=ccrs.PlateCarree(), add_colorbar=False)

opt_anom = dict(
    cmap="RdBu_r", vmin=-2, vmax=2, transform=ccrs.PlateCarree(), add_colorbar=False
)

# ====================
# create axes

f, axs = plt.subplots(1, 5, subplot_kw=dict(projection=ccrs.LambertConformal()))

for ax in axs:
    ax.set_extent([-122, -115, 30, 50], ccrs.PlateCarree())

    ax.add_feature(states, facecolor="none", edgecolor="0.5")
    ax.add_feature(border, facecolor="none", edgecolor="0.5")

# ====================
# plot data

ax = axs[0]
h0 = ds.pi.plot.pcolormesh(ax=ax, cmap="Reds_r", vmin=0, vmax=2, **opt)

ax = axs[1]
levels = np.arange(0.5, 9.5, 1)
cmap, norm = from_levels_and_colors(levels, colors=colors)
h1 = ds.VAC.plot(ax=ax, cmap=cmap, levels=levels, extend="neither", **opt)

ax = axs[2]
h2 = ds.h.plot(ax=ax, **opt_anom)


ax = axs[3]
h3 = ds.TS.plot(ax=ax, **opt_anom)


ax = axs[4]
h4 = ds.LH.plot(ax=ax, **opt_anom)

# ====================
# add colorbars

cbar = mpu.colorbar(h0, axs[0], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks(np.arange(0, 2.1, 1))

cbar = mpu.colorbar(h1, axs[1], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks([1.5, 3.5, 5.5, 7.5])
cbar.set_ticklabels(["a", "b", "c", "d"])

cbar = mpu.colorbar(h2, axs[2], axs[-1], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks(np.arange(-2, 2.1, 0.5))

# ====================
# add labels


# ====================
# add gridlines

## Solution / Final Figure

In [ ]:
opt = dict(transform=ccrs.PlateCarree(), add_colorbar=False)

opt_anom = dict(
    cmap="RdBu_r", vmin=-2, vmax=2, transform=ccrs.PlateCarree(), add_colorbar=False
)

# ====================
# create axes

f, axs = plt.subplots(1, 5, subplot_kw=dict(projection=ccrs.LambertConformal()))

for ax in axs:
    ax.set_extent([-122, -115, 30, 50], ccrs.PlateCarree())

    ax.add_feature(states, facecolor="none", edgecolor="0.5")
    ax.add_feature(border, facecolor="none", edgecolor="0.5")

# ====================
# plot data

ax = axs[0]
h0 = ds.pi.plot.pcolormesh(ax=ax, cmap="Reds_r", vmin=0, vmax=2, **opt)

ax = axs[1]
levels = np.arange(0.5, 9.5, 1)
cmap, norm = from_levels_and_colors(levels, colors=colors)
h1 = ds.VAC.plot(ax=ax, cmap=cmap, levels=levels, extend="neither", **opt)

ax = axs[2]
h2 = ds.h.plot(ax=ax, **opt_anom)


ax = axs[3]
h3 = ds.TS.plot(ax=ax, **opt_anom)


ax = axs[4]
h4 = ds.LH.plot(ax=ax, **opt_anom)

# ====================
# add colorbars

cbar = mpu.colorbar(h0, axs[0], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks(np.arange(0, 2.1, 1))

cbar = mpu.colorbar(h1, axs[1], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks([1.5, 3.5, 5.5, 7.5])
cbar.set_ticklabels(["a", "b", "c", "d"])

cbar = mpu.colorbar(h2, axs[2], axs[-1], size=0.04, pad=0.05, orientation="horizontal")
cbar.set_ticks(np.arange(-2, 2.1, 0.5))

# ====================
# add labels

lbl = "abcde"
for i, ax in enumerate(axs):
    ax.set_title(f"({lbl[i]})", loc="left")

axs[0].set_title(r"$\pi$")
axs[1].set_title("VAC")
axs[2].set_title("")
axs[2].set_title("SH' - SHp'", loc="right")
axs[3].set_title("T'")
axs[4].set_title("LH'")

# ====================
# add gridlines

lon = np.arange(-200, -100, 4)
lat = np.arange(10, 86, 4)

for ax in axs:
    gl = ax.gridlines(ylocs=lat, xlocs=lon, color="0.5", alpha=0.5)


# ====================
# format the figure

f.subplots_adjust(left=0.025, right=0.975, bottom=0.2)
mpu.set_map_layout(ax, 17)

# plt.savefig('ex5_multiple_regional_maps_M_Hauser.png', dpi=300)